In [1]:
import pandas as pd
import dtale
import numpy as np
from sklearn.preprocessing import PolynomialFeatures
import plotly.graph_objects as go # for visualization
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics
import os
import statsmodels.api as sm

import matplotlib.pyplot as plt
%matplotlib inline

%load_ext nb_black
%load_ext blackcellmagic

pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.width', 1000)


<IPython.core.display.Javascript object>

In [2]:
df = pd.read_excel("../../../site_info_ver_3.2.xlsx")


<IPython.core.display.Javascript object>

In [7]:
df = df.loc[df.is_weekend==1]
df.reset_index(drop=True,inplace=True)
df

,Date,Model_number,Site_Name,Israelis_Count,Tourists_Count,Total,region_Central,region_Judea_Samaria,region_North,region_South,is_weekend,operations,is_jewish_holiday,is_jewish_vacation,is_muslims_holiday,is_muslims_vacation,is_camping,special_activity,visit_duration,is_jewishHeritage,is_muslimsHeritage,is_christiansHeritage,is_nationalPark,is_natureReserve,is_archaeology,is_religion,is_animals,is_heritage,is_water,is_cave,is_lookout,pm10,pm2.5,nox,so2,Temperature,is_HeatWave,Tel_Aviv-Yafo_pm10,Jerusalem_pm10,Haifa_pm10,Ashkelon_pm10,Beer-Sheva_pm10,Tel_Aviv-Yafo_pm2.5,Jerusalem_pm2.5,Haifa_pm2.5,Ashkelon_pm2.5,Beer-Sheva_pm2.5,Tel_Aviv-Yafo_nox,Jerusalem_nox,Haifa_nox,Ashkelon_nox,Beer-Sheva_nox,Tel_Aviv-Yafo_so2,Jerusalem_so2,Haifa_so2,Ashkelon_so2,Beer-Sheva_so2,is_Site_exceeded_pm10,is_Site_exceeded_pm2.5,is_Site_exceeded_nox,is_Site_exceeded_so2,Tel_Aviv-Yafo_pm10_exceeded,Jerusalem_pm10_exceeded,Haifa_pm10_exceeded,Ashkelon_pm10_exceeded,Beer-Sheva_pm10_exceeded,Tel_Aviv-Yafo_pm2.5_exceeded,Jerusalem_pm2.5_exceeded,Haifa_pm2.5_exceeded,Ashkelon_pm2.5_exceeded,Beer-Sheva_pm2.5_exceeded,Tel_Aviv-Yafo_so2_exceeded,Jerusalem_so2_exceeded,Haifa_so2_exceeded,Ashkelon_so2_exceeded,Beer-Sheva_so2_exceeded,Tel_Aviv-Yafo_nox_exceeded,Jerusalem_nox_exceeded,Haifa_nox_exceeded,Ashkelon_nox_exceeded,Beer-Sheva_nox_exceeded,Green_border,Season_autumn,Season_spring,Season_summer,Season_winter
0,2016-01-01,5,Alexander Stream-Bet Yannai,7,0,7,1,0,0,0,1,0,0,0,0,0,0,0,3,0,0,0,1,0,0,0,0,0,1,0,0,8.0,NaN,6.20,0.6,15.5,0,64.1,25.6,23.3,11.2,4.9,13.9,30.1,26.2,4.8,2.5,69.1,501.7,90.2,3.8,9.7,7.2,0.1,2.6,1.2,0.9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,1
1,2016-01-02,5,Alexander Stream-Bet Yannai,28,0,28,1,0,0,0,1,0,0,0,0,0,0,0,3,0,0,0,1,0,0,0,0,0,1,0,0,23.5,NaN,66.10,7.5,13.8,0,154.1,277.1,50.6,26.0,51.4,87.1,80.7,93.9,21.6,36.0,749.9,1140.2,842.8,10.8,115.7,16.7,1.9,20.0,8.8,2.3,0,0,1,0,1,1,0,0,0,1,1,1,0,0,0,0,0,0,0,1,1,1,0,1,0,0,0,0,1
2,2016-01-08,5,Alexander Stream-Bet Yannai,14,0,14,1,0,0,0,1,0,0,0,0,0,0,0,3,0,0,0,1,0,0,0,0,0,1,0,0,NaN,NaN,25.80,16.1,20.5,0,80.8,303.2,82.3,45.3,49.3,44.6,73.6,123.1,27.7,25.5,120.8,1152.4,208.7,5.4,37.8,8.0,0.2,2.3,0.8,1.5,0,0,0,0,0,1,0,0,0,1,1,1,0,0,0,0,0,0,0,1,1,1,0,1,0,0,0,0,1
3,2016-01-09,5,Alexander Stream-Bet Yannai,71,0,71,1,0,0,0,1,0,0,0,0,0,0,0,3,0,0,0,1,0,0,0,0,0,1,0,0,36.7,NaN,22.97,0.0,18.2,0,90.9,224.5,24.8,23.7,38.8,49.1,63.5,89.8,26.3,20.2,166.9,1268.1,225.1,2.2,37.1,7.6,0.9,6.3,2.2,2.2,0,0,0,0,0,1,0,0,0,1,1,1,0,0,0,0,0,0,0,1,1,1,0,1,0,0,0,0,1
4,2016-01-15,5,Alexander Stream-Bet Yannai,64,0,64,1,0,0,0,1,0,0,0,0,0,0,0,3,0,0,0,1,0,0,0,0,0,1,0,0,NaN,NaN,57.80,3.9,19.5,0,90.7,105.3,30.0,13.8,13.7,20.3,57.1,25.9,9.7,7.1,338.0,664.9,451.1,4.5,48.3,8.5,0.3,3.2,1.2,1.6,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,1,0,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31472,2021-07-03,2,Baptismal Site Qasr al-Yahud,2,0,2,0,1,0,0,1,0,0,1,0,1,0,0,1,1,0,1,0,0,0,1,0,0,0,0,0,378.5,57.9,720.90,NaN,38.5,1,133.7,378.5,95.0,27.2,30.0,162.8,57.9,109.3,17.7,17.3,1397.4,720.9,356.5,11.4,40.7,9.3,2.9,15.6,2.8,4.4,1,1,1,0,1,1,0,0,0,1,1,1,0,0,0,0,0,0,0,1,1,1,0,1,1,0,0,1,0
31473,2021-07-09,2,Baptismal Site Qasr al-Yahud,5,0,5,0,1,0,0,1,0,0,1,0,1,0,0,1,1,0,1,0,0,0,1,0,0,0,0,0,30.5,7.5,2.60,2.4,38.5,1,121.6,117.9,102.0,29.9,25.6,138.0,47.5,109.6,17.2,11.9,111.7,49.1,60.3,1.5,15.6,8.7,2.9,8.5,2.0,3.7,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,1,1,1,0,0,1,0,0,1,0
31474,2021-07-10,2,Baptismal Site Qasr al-Yahud,8,0,8,0,1,0,0,1,0,0,1,0,1,0,0,1,1,0,1,0,0,0,1,0,0,0,0,0,31.8,12.5,26.90,NaN,38.5,1,138.2,431.3,115.1,41.1,40.8,118.9,77.5,110.0,23.8,12.1,1118.4,902.9,415.0,5.7,46.1,13.0,3.6,17.8,1.8,5.9,0,0,0,0,1,1,0,0,0

<IPython.core.display.Javascript object>

In [8]:
df_sites = []
sites = df.Site_Name.unique()
for site in sites:
    tmp = df[df.Site_Name==site]
    tmp.reset_index(drop=True,inplace=True)
    df_sites.append(tmp)

396
547
548
544
550
480
546
547
513
550
543
544
544
549
547
550
547
543
530
539
541
550
550
550
531
533
524
530
545
537
533
543
253
538
530
535
531
548
508
478
548
548
547
529
541
508
526
545
502
460
542
541
515
537
504
494
453
533
428
148
387
96


<IPython.core.display.Javascript object>

Check Null

In [9]:
# df.isna().sum()
# df=df.dropna()
# df.reset_index(inplace=True,drop=True)
for d in df_sites:
    d.drop(['pm10','nox','so2','pm2.5',"Site_Name"],axis=1,inplace=True)
# len(df[df.isnull().any(axis=1)])/len(df)

<IPython.core.display.Javascript object>

Defiend X and y

In [10]:
X_site = []
targets_site = []
for d in df_sites:
    X_site.append(d.drop(['Israelis_Count','Tourists_Count','Total',"Date","Model_number"],axis=1))
    targets_site.append(d[['Israelis_Count','Tourists_Count','Total']])

<IPython.core.display.Javascript object>

Scaleing

In [11]:
from sklearn.preprocessing import MinMaxScaler
scaled_X_sites=[]
scaled_targets_sites=[]
for i in range(len(X_site)):
    #scale X
    scaler = MinMaxScaler()
    scaled_X_sites.append( pd.DataFrame(scaler.fit_transform(X_site[i]), columns=X_site[i].columns))

    #scale y
    scaled_targets_sites.append( np.log(targets_site[i]+0.01))

<IPython.core.display.Javascript object>

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import RFE
from sklearn.model_selection import GridSearchCV

j=0 #############
targets = [scaled_targets_sites[j].Israelis_Count,scaled_targets_sites[j].Total,scaled_targets_sites[j].Tourists_Count] # get target info of this site
targets_list_name = ['Israelis_Count','Total','Tourists_Count']
idx=0
for y in targets:
    X=scaled_X_sites[j] # get X info of this site
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=312148513)
    train_df_scaled = pd.merge(left=X_train, right=y_train, left_index=True, right_index=True)
    test_df_scaled = pd.merge(left=X_test, right=y_test, left_index=True, right_index=True)
    # step-1: create a cross-validation scheme
    folds = KFold(n_splits = 25, shuffle = True, random_state = 100)
    
    # step-2: specify range of hyperparameters to tune
    hyper_params = [{'n_features_to_select': list(range(0, len(X.columns)))}]
    
    
    # step-3: perform grid search
    # 3.1 specify model
    lm = LinearRegression()
    lm.fit(X_train, y_train)
    rfe = RFE(lm)             
    
    # 3.2 call GridSearchCV()
    model_cv = GridSearchCV(estimator = rfe, 
                            param_grid = hyper_params, 
                            scoring= 'r2', 
                            cv = folds, 
                            verbose = 1,
                            return_train_score=True)      
    
    # fit the model
    model_cv.fit(X_train, y_train)   
    #######################################################################################################################
    #train
    fitted = np.exp(model_cv.predict(X_train))
    predicted_train = round(pd.Series(fitted, index=y_train.index, name='Predicted_train_'+targets_list_name[idx]),ndigits=2)
    orignal_train=np.exp(train_df_scaled[[targets_list_name[idx]]])
    train_df = pd.merge(left=orignal_train, right=predicted_train, left_index=True, right_index=True)
    train_df
    #test
    fitted = np.exp(model_cv.predict(X_test))
    predicted_test = round(pd.Series(fitted, index=y_test.index, name='Predicted_test_'+targets_list_name[idx]),ndigits=2)
    orignal_test = np.exp(test_df_scaled[[targets_list_name[idx]]])
    test_df = pd.merge(left=orignal_test, right=predicted_test, left_index=True, right_index=True)
    test_df

    MSE_Training = metrics.mean_squared_error(train_df[targets_list_name[idx]], train_df['Predicted_train_'+targets_list_name[idx]])
    RMSE_Training = np.sqrt(metrics.mean_squared_error(train_df[targets_list_name[idx]], train_df['Predicted_train_'+targets_list_name[idx]]))
    MAE_Training = metrics.mean_absolute_error(train_df[targets_list_name[idx]], train_df['Predicted_train_'+targets_list_name[idx]])
    R2_Training=metrics.r2_score(train_df[targets_list_name[idx]], train_df['Predicted_train_'+targets_list_name[idx]])
    std_Training = np.std(predicted_train)
    # f.write("------ TRAIN DATA ------\n")
    # #f.write("Model "+' mod0' +" Accuracy: "+str(ACC_Training)+"\n")
    # f.write("MSE : "+str(MSE_Training)+", RMSE: "+str(RMSE_Training)+", MAE : "+str(MAE_Training)+"\n")
    # f.write("R2 TRAIN "+ str(R2_Training)+"\n")
   #  correct_rows = test_df.loc[abs(test_df['Predicted_test_'+targets_list_name[idx]]-test_df[targets_list_name[idx]])<=50].any(axis=1).count()
    #  ACC_Test = round(correct_rows/len(test_df),3)
    MSE_Test = metrics.mean_squared_error(test_df[targets_list_name[idx]], test_df['Predicted_test_'+targets_list_name[idx]])
    RMSE_Test = np.sqrt(metrics.mean_squared_error(test_df[targets_list_name[idx]], test_df['Predicted_test_'+targets_list_name[idx]]))
    MAE_Test = metrics.mean_absolute_error(test_df[targets_list_name[idx]], test_df['Predicted_test_'+targets_list_name[idx]])
    R2_Test = metrics.r2_score(test_df[targets_list_name[idx]], test_df['Predicted_test_'+targets_list_name[idx]])
    std_test = np.std(predicted_test)
    idx+=1

    print('rmse_train', RMSE_Training)
    print('rmse_test', RMSE_Test)
    print('r2_train', R2_Training)
    print('r2_test', R2_Test)

Model MLR with Pollutions droped Null rows

In [33]:
Model_type= "MLR"
res=pd.read_excel("../../../res.xlsx")


for j in range(len(scaled_X_sites)):
  Descripton = "Without Pollutions Without outliers only weekend for each site " + sites[j]
  print(sites[j])
  targets = [scaled_targets_sites[j].Israelis_Count,scaled_targets_sites[j].Total,scaled_targets_sites[j].Tourists_Count] # get target info of this site
  targets_list_name = ['Israelis_Count','Total','Tourists_Count']
  idx=0
  for y in targets:

      #split the data, train the model and get prediction for the training and for the test  
      #######################################################################################################################
      X=scaled_X_sites[j] # get X info of this site

      X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=312148513)
      train_df_scaled = pd.merge(left=X_train, right=y_train, left_index=True, right_index=True)
      test_df_scaled = pd.merge(left=X_test, right=y_test, left_index=True, right_index=True)

      lm = LinearRegression()  # define our model using least square method
      # step-1: create a cross-validation scheme
      folds = KFold(n_splits = 12, shuffle = True, random_state =316173897 )

      # step-2: specify range of hyperparameters to tune
      hyper_params = [{'n_features_to_select': list(range(0, len(X.columns)))}]


      # step-3: perform grid search
      # 3.1 specify model
      lm = LinearRegression()
      lm.fit(X_train, y_train)
      rfe = RFE(lm)             

      # 3.2 call GridSearchCV()
      model_cv = GridSearchCV(estimator = rfe, 
                              param_grid = hyper_params, 
                              scoring= 'r2', 
                              cv = folds, 
                              verbose = 1,
                              return_train_score=True)      

      # fit the model
      model_cv.fit(X_train, y_train) 

      #train
      fitted = np.exp(model_cv.predict(X_train))
      predicted_train = round(pd.Series(fitted, index=y_train.index, name='Predicted_train_'+targets_list_name[idx]),ndigits=2)
      orignal_train=np.exp(train_df_scaled[[targets_list_name[idx]]])
      train_df = pd.merge(left=orignal_train, right=predicted_train, left_index=True, right_index=True)
      train_df

      #test
      fitted = np.exp(model_cv.predict(X_test))
      predicted_test = round(pd.Series(fitted, index=y_test.index, name='Predicted_test_'+targets_list_name[idx]),ndigits=2)
      orignal_test = np.exp(test_df_scaled[[targets_list_name[idx]]])
      test_df = pd.merge(left=orignal_test, right=predicted_test, left_index=True, right_index=True)
      test_df
       #######################################################################################################################





       #plot the residuals graph
       #######################################################################################################################

       #calculate the residuals
      test_df['residuals'] = test_df['Predicted_test_'+targets_list_name[idx]] - test_df[targets_list_name[idx]]
      train_df['residuals'] = train_df['Predicted_train_'+targets_list_name[idx]] - train_df[targets_list_name[idx]]

      fig= go.Figure()
      fig.add_trace(
        go.Scatter(
            x=train_df['Predicted_train_'+targets_list_name[idx]],
            y=train_df.residuals,
            mode='markers',
            name='train residuals',
            marker_color='blue',
            marker_size=1.5,
            marker_line_width=0,
         )
       )
      fig.add_trace(
        go.Scatter(
            x=test_df['Predicted_test_'+targets_list_name[idx]],
            y=test_df.residuals,
            mode='markers',
            name='test residuals',
            marker_color='red',
            marker_size=1.5,
            marker_line_width=0,
         )
       )
      fig.add_trace(
         go.Scatter(
            x=test_df['Predicted_test_'+targets_list_name[idx]],
            y=test_df.residuals*0,
            mode='lines',
            name='zero line',
            marker_color='black',
            marker_size=1.5,
            marker_line_width=0,
         )
       )
      fig.update_layout(
          title="Residuals of Predicted "+targets_list_name[idx],
          xaxis_title="Predicted "+targets_list_name[idx],
          yaxis_title="Residuals",
          font=dict(
              size=14,
              color="RebeccaPurple"
          )
      )
      # fig.show()
       #######################################################################################################################

       #create folder
       ########################################
      if not os.path.exists(sites[j]):
        os.mkdir(sites[j])

      if not os.path.exists(sites[j]+'/'+Descripton):
        os.mkdir(sites[j]+'/'+Descripton)
      fig.write_image(sites[j]+'/'+Descripton+"/results_"+targets_list_name[idx]+'_.png', width=1500, height=600)
      #########################################

      #create txt file with the metrix result 
      ################################################################################################################################################     
      f = open(sites[j]+'/'+Descripton+"/results_"+targets_list_name[idx]+'_.txt', 'w')

      #  correct_rows = train_df.loc[abs(train_df['Predicted_train_'+targets_list_name[idx]]-train_df[targets_list_name[idx]])<=50].any(axis=1).count()
      #  ACC_Training = round(correct_rows/len(train_df),3)

      if len(test_df.loc[test_df['Predicted_test_'+targets_list_name[idx]]>=np.infty])/len(test_df)*100 <1:
          test_df.drop(test_df.loc[test_df['Predicted_test_'+targets_list_name[idx]]>=np.infty].index,inplace=True)
      MSE_Training = metrics.mean_squared_error(train_df[targets_list_name[idx]], train_df['Predicted_train_'+targets_list_name[idx]])
      RMSE_Training = np.sqrt(metrics.mean_squared_error(train_df[targets_list_name[idx]], train_df['Predicted_train_'+targets_list_name[idx]]))
      MAE_Training = metrics.mean_absolute_error(train_df[targets_list_name[idx]], train_df['Predicted_train_'+targets_list_name[idx]])
      R2_Training=metrics.r2_score(train_df[targets_list_name[idx]], train_df['Predicted_train_'+targets_list_name[idx]])
      std_Training = np.std(predicted_train)

      f.write("------ TRAIN DATA ------\n")
      #f.write("Model "+' mod0' +" Accuracy: "+str(ACC_Training)+"\n")
      f.write("MSE : "+str(MSE_Training)+", RMSE: "+str(RMSE_Training)+", MAE : "+str(MAE_Training)+"\n")
      f.write("R2 TRAIN "+ str(R2_Training)+"\n")


      #  correct_rows = test_df.loc[abs(test_df['Predicted_test_'+targets_list_name[idx]]-test_df[targets_list_name[idx]])<=50].any(axis=1).count()
      #  ACC_Test = round(correct_rows/len(test_df),3)

      MSE_Test = metrics.mean_squared_error(test_df[targets_list_name[idx]], test_df['Predicted_test_'+targets_list_name[idx]])
      RMSE_Test = np.sqrt(metrics.mean_squared_error(test_df[targets_list_name[idx]], test_df['Predicted_test_'+targets_list_name[idx]]))
      MAE_Test = metrics.mean_absolute_error(test_df[targets_list_name[idx]], test_df['Predicted_test_'+targets_list_name[idx]])
      R2_Test = metrics.r2_score(test_df[targets_list_name[idx]], test_df['Predicted_test_'+targets_list_name[idx]])
      std_test = np.std(predicted_test)

      f.write("\n")
      f.write("------ TEST DATA ------\n")
      #f.write("Model "+' mod0' +" Accuracy: "+str(ACC_Test)+"\n")
      f.write("MSE : "+str(MSE_Test)+", RMSE: "+str(RMSE_Test)+", MAE : "+str(MAE_Test)+"\n")
      f.write("R2 TEST "+ str(R2_Test)+"\n")
      f.write("--------------------------------\n")

      f.close()
      ################################################################################################################################################     

      #Add to excel result the new line
      ################################################################################################################################################
      Target=targets_list_name[idx]
      Site_in_this_model=sites[j]
      new_row = {
        'Descripton':Descripton,
        "Target":Target,
        'Model_type':Model_type,
        # 'Model_number':Model_number,
        'Site_in_this_model':Site_in_this_model,
        "Size Train":len(train_df),
        # 'ACC_Training':ACC_Training,
        'MAE_Training':MAE_Training,
        'MSE_Training':MSE_Training,
        'RMSE_Training':RMSE_Training,
        'STD_Training':std_Training,
        'R2_Training':R2_Training,
        "Size Test":len(test_df),
        # 'ACC_Test':ACC_Test,
        'MAE_Test':MAE_Test,
        'MSE_Test':MSE_Test,
        'RMSE_Test':RMSE_Test,
        'STD_Test':std_test,
        'R2_Test':R2_Test 
        }

      res = res.append(new_row,ignore_index=True)
      ################################################################################################################################################     

      # Index for the next target y
      idx+=1


res.to_excel("../../../res.xlsx",index=False)


Alexander Stream-Bet Yannai
Fitting 12 folds for each of 76 candidates, totalling 912 fits
Fitting 12 folds for each of 76 candidates, totalling 912 fits
Fitting 12 folds for each of 76 candidates, totalling 912 fits
Apollonia
Fitting 12 folds for each of 76 candidates, totalling 912 fits
Fitting 12 folds for each of 76 candidates, totalling 912 fits
Fitting 12 folds for each of 76 candidates, totalling 912 fits
Arbel
Fitting 12 folds for each of 76 candidates, totalling 912 fits
Fitting 12 folds for each of 76 candidates, totalling 912 fits
Fitting 12 folds for each of 76 candidates, totalling 912 fits
Avdat
Fitting 12 folds for each of 76 candidates, totalling 912 fits
Fitting 12 folds for each of 76 candidates, totalling 912 fits
Fitting 12 folds for each of 76 candidates, totalling 912 fits
Ayun Stream
Fitting 12 folds for each of 76 candidates, totalling 912 fits
Fitting 12 folds for each of 76 candidates, totalling 912 fits
Fitting 12 folds for each of 76 candidates, totalling 91

<IPython.core.display.Javascript object>

In [64]:
# MSE_Test = metrics.mean_squared_error(test_df[targets_list_name[idx]], test_df['Predicted_test_'+targets_list_name[idx]])

if len(test_df.loc[test_df['Predicted_test_'+targets_list_name[idx]]>=np.infty])/len(test_df)*100 <1:
          test_df.drop(test_df.loc[test_df['Predicted_test_'+targets_list_name[idx]]>=np.infty].index)

<IPython.core.display.Javascript object>